In [1]:
# %pylab

import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
from mpl_toolkits.mplot3d import Axes3D
np = numpy
plt = pyplot

from pylab import *
from numpy import *

In [21]:
def beta_continue(n,a,q,beta_guess): # foireux
    C1 = 0
    C2 = 0
    for k in range(n,0,-1):
        C1 = q**2/((beta_guess+2*n)**2 - a - C1)
        C2 = q**2/((beta_guess-2*n)**2 - a - C2)
        
    return a + C1 + C2

In [243]:
def beta_continue_alamano(a,q,beta_guess): #a l'air ok
    C1 = q**2/( (beta_guess+2)**2 - a - q**2/( (beta_guess+4)**2 - a - q**2/( (beta_guess+6)**2 - a - q**2/( (beta_guess+8)**2 - a - q**2/( (beta_guess+10)**2 - a ) ) ) ) )
    C2 = q**2/( (beta_guess-2)**2 - a - q**2/( (beta_guess-4)**2 - a - q**2/( (beta_guess-6)**2 - a - q**2/( (beta_guess-8)**2 - a - q**2/( (beta_guess-10)**2 - a  ) ) ) ) )
    return a + C1 + C2

In [272]:
beta_guess = 0.402
a = 2
q = 0.16
for k in range(15):
    beta_guess = beta_continue_alamano(a,q,beta_guess)
    beta_guess = sqrt(beta_guess)
#     print(beta_guess)
print('non adiab',beta_guess)
print('adiab    ',sqrt(a+q**2/2))

print('non adiab',0.5*beta_guess*Omega/1000/2/pi)
print('adiab    ',0.5*sqrt(a+q**2/2)*Omega/1000/2/pi)

non adiab 1.4096842905306817
adiab     1.418731828077456
non adiab 1409.6842905306817
adiab     1418.7318280774557


In [164]:
beta_guess = 0.5
delta_beta = 10
a = 0
q = 0.8
while abs(delta_beta) > 0.0001:
    delta_beta = beta_guess**2 - beta_continue(1,a,q,beta_guess) # 0.908 vs 0.9505
    if delta_beta < 0:
        beta_guess = beta_guess*1.1
    else:
        beta_guess = beta_guess*0.9
    
print(beta_guess)
print(sqrt(a+q**2/2))

0.7403033783520424
0.5656854249492381


In [212]:
beta_guess = 0.07
delta_beta = 10
a = 0
q = 0.1
while abs(delta_beta) > 0.005:
    delta_beta = beta_guess**2 - beta_continue_alamano(a,q,beta_guess) # 0.908 vs 0.9505
    if delta_beta < 0:
        beta_guess = beta_guess*1.1
    else:
        beta_guess = beta_guess*0.9
    
print(beta_guess)
print(sqrt(a+q**2/2))

0.07700000000000001
0.07071067811865477


In [2]:
# Constantes de la physique
# ABSOLUMENT RECHARGER APRÈS AVOIR EXECUTÉ LES CASES D'IMPORT AU DESSUS

C_e = 1.602e-19        # Coulomb
kb = 1.38064852*1e-23  # Boltzman
m_Ca = 40.078*1.66054e-27 # masse Ca 40.078
m_GM = 1e6*1.66054e-27 # la masse de la GMol
eps0 = 8.854187*1e-12  # permittivité électrique du vide

r0 = 2.5e-3 # 2.5e-3   # rayon piège Ca+
d0 = 4e-3/2            # longueur piège Ca+

Omega = 2.00e6*2*pi # 2.047e6

bk = 4 # nombre de barreaux par groupe (2 -> 4-pole , 4 -> 8-pole ...)

mkappa = 0.23  # écrantage piège réel GiantMol
wzLC = (2*pi*90806.9982303)**2
kappa_simion = m_Ca*d0**2*wzLC/(2*C_e)
print('%s = %f' % ('$\kappa_{simion}$',kappa_simion) )

zeta = kappa_simion*r0**2/d0**2

$\kappa_{simion}$ = 0.270471


In [3]:
N_ions = 1024

# for k,l in enumerate(list(points_and_coord)):
my_udc = 2
my_urf = 64.61
q = 4 * C_e * my_urf / (m_Ca*r0**2*Omega**2)
# omegaz = sqrt( 2 * C_e * my_udc / (m_Ca*d0**2) )
# omegax_2.append(Omega**2 / 4 *  q**2/2)
# beta2 = q**2/(2-q**2) - 7*q**4/(32*4) + 29*q**6/(64*4*9)
# omegax_2bis.append(0.25*Omega**2 * beta2 )
# omegar_2 = Omega**2 / 4 * ( q**2/2 - 0.5*(omegaz/Omega)**2 )

In [4]:
a_u = 0
q_u = q